In [1]:
import pandas as pd
import numpy as np
import nltk
import tensorflow as tf
import tensorflow_addons as tfa
import keras
import torchtext

import nlpaug
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.sentence as nas
import nlpaug.flow as nafc
from nlpaug.util import Action


from sklearn.preprocessing import LabelEncoder
seed = 2000
np.random.seed(seed)
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense, Embedding
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence

from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from focal_loss import SparseCategoricalFocalLoss
import torch
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, AdamW

import warnings
warnings.filterwarnings("ignore")

2023-01-27 21:36:21.903543: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/nitanshjain/.local/share/virtualenvs/Sem_Eval-qQJYuRaW/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_sub1_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/preprocessed_data/subtask1/en_train_subtask_1.csv')
print(train_sub1_df.shape)
train_sub1_df.head()
print(train_sub1_df.genre.value_counts())

final_train_sub1_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/preprocessed_data/subtask1/final_train_subtask_1.csv')
print(final_train_sub1_df.shape)
final_train_sub1_df.head()
print(final_train_sub1_df.genre.value_counts())

dev_sub1_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/preprocessed_data/subtask1/en_dev_subtask_1.csv')
print(dev_sub1_df.shape)
dev_sub1_df.head()
print(dev_sub1_df.genre.value_counts())

# test_sub1_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/preprocessed_data/extra_test.csv')
# print(test_sub1_df.shape)

(433, 10)
opinion      382
reporting     41
satire        10
Name: genre, dtype: int64
(1437, 11)
opinion      1073
reporting     271
satire         93
Name: genre, dtype: int64
(83, 10)
reporting    54
opinion      20
satire        9
Name: genre, dtype: int64


In [3]:
le = LabelEncoder()
train_sub1_df['genre'] = le.fit_transform(train_sub1_df['genre'])
print(train_sub1_df.genre.value_counts())

le = LabelEncoder()
final_train_sub1_df['genre'] = le.fit_transform(final_train_sub1_df['genre'])
print(final_train_sub1_df.genre.value_counts())

le = LabelEncoder()
dev_sub1_df['genre'] = le.fit_transform(dev_sub1_df['genre'])
print(dev_sub1_df.genre.value_counts())
dev_sub1_df.head()

0    382
1     41
2     10
Name: genre, dtype: int64
0    1073
1     271
2      93
Name: genre, dtype: int64
1    54
0    20
2     9
Name: genre, dtype: int64


,id,genre,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,820791520,1,George III Lost America.\n,Theresa May Could Lose the United Kingdom Over...,george iii lose america,"[(george, 'NNP'), (iii, 'NNP'), (lose, 'VB'), ...","[(george iii, 'PERSON', 380), (america, 'GPE',...",theresa may could lose the united kingdom over...,"[(theresa, 'NN'), (may, 'MD'), (could, 'MD'), ...","[(the united kingdom, 'GPE', 384), (the europe..."
1,821040551,1,Queen Elizabeth Would Be Evacuated in Event of...,If Britain leaves the European Union without a...,queen elizabeth would be evacuate in event of ...,"[(queen, 'NNP'), (elizabeth, 'NNP'), (would, '...","[(elizabeth, 'PERSON', 380), (brexit riot repo...",if britain leave the european union without tr...,"[(if, 'IN'), (britain, 'NNP'), (leave, 'VBP'),...","[(britain, 'GPE', 384), (the european union, '..."
2,813552066,1,"You insult us, ambassador: Woody Johnson flagr...",With three months until Britain leaves the Eur...,you insult us ambassador woody johnson flagran...,"[(you, 'PRP'), (insult, 'VBP'), (us, 'NNP'), (...","[(woody johnson, 'PERSON', 380), (peter, 'PERS...",with three month until britain leave the europ...,"[(with, 'IN'), (three, 'CD'), (month, 'NN'), (...","[(three month, 'DATE', 391), (britain, 'GPE', ..."
3,817176202,1,"The British People, as Well as the Politicians...",The British Parliament just handed Prime Minis...,the british people as well as the politician d...,"[(the, 'DT'), (british, 'JJ'), (people, 'NNS')...","[(british, 'NORP', 381)]",the british parliament just hand prime ministe...,"[(the, 'DT'), (british, 'JJ'), (parliament, 'N...","[(british, 'NORP', 381), (british, 'NORP', 381..."
4,820419869,1,No break from Brexit: RT takes a look at lates...,As British MPs are told that their February br...,no break from brexit rt take look at late deve...,"[(no, 'DT'), (break, 'NN'), (from, 'IN'), (bre...","[(brexit rt, 'ORG', 383)]",as british mp be tell that their february brea...,"[(as, 'IN'), (british, 'NNP'), (mp, 'NNP'), (b...","[(british, 'NORP', 381), (february, 'DATE', 39..."


In [4]:
n0 = 914
n1 = 118
final_train_sub1_df = final_train_sub1_df.drop(final_train_sub1_df[final_train_sub1_df['genre'].eq(0)].sample(n0).index)
final_train_sub1_df = final_train_sub1_df.drop(final_train_sub1_df[final_train_sub1_df['genre'].eq(1)].sample(n1).index)
print(final_train_sub1_df.genre.value_counts())
final_train_sub1_df.shape

0    159
1    153
2     93
Name: genre, dtype: int64


(405, 11)

In [5]:
new_train_df = pd.DataFrame(final_train_sub1_df.loc[final_train_sub1_df['genre']==0])
new_train_df = new_train_df.append(final_train_sub1_df.loc[final_train_sub1_df['genre']==1])
new_train_df = new_train_df.append(final_train_sub1_df.loc[final_train_sub1_df['genre']==2])
new_train_df.genre.value_counts()


0    159
1    153
2     93
Name: genre, dtype: int64

In [6]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight(
    class_weight = "balanced",
    classes = np.unique(new_train_df.genre),
    y = new_train_df.genre
)

class_weights = dict(zip(np.unique(new_train_df.genre), class_weights))
class_weights

{0: 0.8490566037735849, 1: 0.8823529411764706, 2: 1.4516129032258065}

In [7]:
# aug = naw.SynonymAug(aug_src='wordnet',aug_max=3)
aug = naw.ContextualWordEmbsAug(model_path='bert-base-uncased', action="substitute", aug_max=3)

In [8]:
x_train = new_train_df.preprocessed_articles + new_train_df.preprocessed_headlines
y_train = new_train_df[['genre']]

x_dev = dev_sub1_df.preprocessed_articles + dev_sub1_df.preprocessed_headlines
y_dev = dev_sub1_df[['genre']]

# x_train = new_train_df.text
# y_train = new_train_df.target

# x_test = new_train_df.text
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', num_labels=3, output_attentions=True)

In [9]:
augmented_sentences=[]
augmented_sentences_labels=[]
count = 0
for i in x_train.index:
    count+=1
    if count%10==0:
        print("Running count = ", count)
    if y_train.genre[i]==0:
        try:
            temps=aug.augment(x_train[i], n=2)
            for sent in temps:
                augmented_sentences.append(sent)
                augmented_sentences_labels.append(0)
        except:
            continue
    if y_train.genre[i]==1:
        try:
            temps=aug.augment(x_train[i], n=2)
            for sent in temps:
                augmented_sentences.append(sent)
                augmented_sentences_labels.append(1)
        except:
            continue
    if y_train.genre[i]==2:
        try:
            temps=aug.augment(x_train[i], n=2)
            for sent in temps:
                augmented_sentences.append(sent)
                augmented_sentences_labels.append(2)
        except:
            continue

Running count =  10
Running count =  20
Running count =  30
Running count =  40
Running count =  50
Running count =  60
Running count =  70
Running count =  80
Running count =  90
Running count =  100
Running count =  110
Running count =  120
Running count =  130
Running count =  140
Running count =  150
Running count =  160
Running count =  170
Running count =  180
Running count =  190
Running count =  200
Running count =  210
Running count =  220
Running count =  230
Running count =  240
Running count =  250
Running count =  260
Running count =  270
Running count =  280
Running count =  290
Running count =  300
Running count =  310
Running count =  320
Running count =  330
Running count =  340
Running count =  350
Running count =  360
Running count =  370
Running count =  380
Running count =  390
Running count =  400


In [10]:
x_train_aug = pd.Series()
y_train_aug = pd.Series()
x_train_aug = x_train_aug.append(pd.Series(augmented_sentences), ignore_index=True)
y_train_aug = y_train_aug.append(pd.Series(augmented_sentences_labels), ignore_index=False)

In [11]:
x_train = x_train_aug.to_numpy().reshape(-1)
y_train = y_train_aug.to_numpy().reshape(-1,1)
x_dev = x_dev.to_numpy().reshape(-1)
y_dev = y_dev.to_numpy().reshape(-1,1)


In [12]:
def roberta_encode(texts, tokenizer):
    MAX_LEN = 512
    ct = len(texts)
    input_ids = np.ones((ct, MAX_LEN), dtype='int32')
    attention_mask = np.zeros((ct, MAX_LEN), dtype='int32')
    token_type_ids = np.zeros((ct, MAX_LEN), dtype='int32') # Not used in text classification

    for k, text in enumerate(texts):
        # Tokenize
        tok_text = tokenizer.tokenize(str(text))
        
        # Truncate and convert tokens to numerical IDs
        enc_text = tokenizer.convert_tokens_to_ids(tok_text[:(MAX_LEN-2)])
        
        input_length = len(enc_text) + 2
        input_length = input_length if input_length < MAX_LEN else MAX_LEN
        
        # Add tokens [CLS] and [SEP] at the beginning and the end
        input_ids[k,:input_length] = np.asarray([0] + enc_text + [2], dtype='int32')
        
        # Set to 1s in the attention input
        attention_mask[k,:input_length] = 1

    return {
        'input_word_ids': input_ids,
        'input_mask': attention_mask,
        'input_type_ids': token_type_ids
    }

In [13]:
x_train = roberta_encode(x_train, tokenizer)
x_dev = roberta_encode(x_dev, tokenizer)

y_train = np.asarray(y_train, dtype='int32')
y_dev = np.asarray(y_dev, dtype='int32')

In [14]:
def build_model(n_categories):
    MAX_LEN = 512
    input_word_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_word_ids')
    input_mask = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_mask')
    input_type_ids = tf.keras.Input(shape=(MAX_LEN,), dtype=tf.int32, name='input_type_ids')
    
    

    # Import RoBERTa model from HuggingFace
    roberta_model = TFRobertaForSequenceClassification.from_pretrained('roberta-base')
    # custom_objects = {"TFRobertaForSequenceClassification": TFRobertaForSequenceClassification}
    # config = roberta_model_copy.get_config()
    # with tf.keras.utils.custom_object_scope(custom_objects):
    #     roberta_model = TFRobertaForSequenceClassification.from_config(config)
    x = roberta_model(input_word_ids, attention_mask=input_mask, token_type_ids=input_type_ids)

    # Huggingface transformers have multiple outputs, em
    # beddings are the first one,
    # so let's slice out the first position
    x = x[0]

    x = tf.keras.layers.Dropout(rate=0.1)(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(256, activation='relu')(x)
    x = tf.keras.layers.Dense(n_categories, activation='softmax')(x)

    model = tf.keras.Model(inputs=[input_word_ids, input_mask, input_type_ids], outputs=x)
    model.compile(
        optimizer=tf.keras.optimizers.experimental.Adam(learning_rate=0.00001),
        loss=SparseCategoricalFocalLoss(gamma=2),
        metrics=['accuracy'])
    
    

    return model

In [15]:
model_copy = build_model(3)
model_copy.summary()

2023-01-27 22:09:49.960942: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_word_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 input_mask (InputLayer)        [(None, 512)]        0           []                               
                                                                                                  
 input_type_ids (InputLayer)    [(None, 512)]        0           []                               
                                                                                                  
 tf_roberta_for_sequence_classi  TFSequenceClassifie  124647170  ['input_word_ids[0][0]',         
 fication (TFRobertaForSequence  rOutput(loss=None,               'input_mask[0][0]',         

In [16]:
tf.config.run_functions_eagerly(True)

In [27]:
batch_size = 16

checkpoint_filepath = '/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/codefiles/subtask1/roberta/weights/weights-improvement-multimodal-h5-{epoch:02d}-{val_accuracy:.2f}.h5'

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy', 
    save_best_only=False,
    save_weights_only=False,
    mode='max'
    )

reduce_lr_callback = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_accuracy', factor=0.2,
                              patience=10, min_lr=0.0001)

model_copy.fit(x=x_train,
                y=y_train,
                batch_size=batch_size,
                epochs=10,
                callbacks=[model_checkpoint_callback, reduce_lr_callback],
                validation_data=(x_dev, y_dev),
                shuffle=True,
                verbose=1, class_weight=class_weights
                )
        

Epoch 1/5
51/51 [==============================] - 7088s 139s/step - loss: 0.2742 - accuracy: 0.7512 - val_loss: 0.5242 - val_accuracy: 0.3735
Epoch 2/5
51/51 [==============================] - 7055s 138s/step - loss: 0.2900 - accuracy: 0.7201 - val_loss: 0.4040 - val_accuracy: 0.5060
Epoch 3/5
51/51 [==============================] - 7031s 138s/step - loss: 0.1759 - accuracy: 0.8308 - val_loss: 0.4270 - val_accuracy: 0.5783
Epoch 4/5
51/51 [==============================] - 7039s 138s/step - loss: 0.1358 - accuracy: 0.8657 - val_loss: 0.4811 - val_accuracy: 0.5663
Epoch 5/5
51/51 [==============================] - 7005s 137s/step - loss: 0.0969 - accuracy: 0.9129 - val_loss: 0.5203 - val_accuracy: 0.5663
Epoch 1/5
51/51 [==============================] - 6980s 137s/step - loss: 0.0688 - accuracy: 0.9303 - val_loss: 0.7421 - val_accuracy: 0.4096
Epoch 2/5
15/51 [=======>......................] - ETA: 1:32:27 - loss: 0.0856 - accuracy: 0.9083

In [ ]:
from keras.models import load_model

with tf.keras.utils.CustomObjectScope({'TFRobertaForSequenceClassification': TFRobertaForSequenceClassification.from_pretrained('roberta-base')}):
    model = load_model('/content/weights-improvement-multimodal-h5-01-0.57.h5')  
model.summary(print_fn=print)


In [ ]:
y_pred = model_copy.predict(x_dev)
print(y_pred)

3/3 [==============================] - 84s 24s/step
[[0.46505716 0.27893054 0.25601232]
 [0.26064035 0.5052562  0.23410344]
 [0.4774259  0.3284097  0.1941644 ]
 [0.43730757 0.3376547  0.22503771]
 [0.25886112 0.5445365  0.1966024 ]
 [0.47891247 0.28328687 0.23780061]
 [0.31777605 0.47541195 0.20681195]
 [0.35552657 0.3245846  0.3198889 ]
 [0.321076   0.38235837 0.29656565]
 [0.25898972 0.544331   0.19667925]
 [0.30466628 0.51337916 0.18195464]
 [0.31616122 0.48571527 0.19812343]
 [0.43953463 0.270253   0.2902123 ]
 [0.2839397  0.47513548 0.24092482]
 [0.36571234 0.45796764 0.17632   ]
 [0.36682653 0.45377678 0.17939678]
 [0.2602805  0.5375579  0.2021616 ]
 [0.26298368 0.46841362 0.2686027 ]
 [0.2958601  0.5178372  0.18630266]
 [0.3151076  0.5044825  0.18040988]
 [0.29034206 0.50789446 0.20176345]
 [0.33625668 0.48749006 0.17625321]
 [0.33169404 0.4872298  0.1810761 ]
 [0.42655975 0.36401898 0.20942126]
 [0.2863524  0.5267157  0.18693194]
 [0.27097887 0.53918374 0.18983741]
 [0.26433808

In [25]:
y_pred_final = list()
y_pred_sub = list()
for val in y_pred:
    if val[0]>val[1] and val[0]>val[2]:
        y_pred_final.append(0)
        y_pred_sub.append('opinion')
    elif val[1]>val[0] and val[1]>val[2]:
        y_pred_final.append(1)
        y_pred_sub.append('reporting')
    else:
        y_pred_final.append(2)
        y_pred_sub.append('satire')

print(y_pred_final) 
print(y_dev) 
# print(list(y_dev))  
    

[0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 2, 0, 1, 0, 1, 0, 1, 1, 0, 0, 2, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0]
[[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [0]
 [1]
 [1]
 [0]
 [1]
 [0]
 [1]
 [2]
 [1]
 [0]
 [1]
 [2]
 [0]
 [0]
 [0]
 [1]
 [1]
 [2]
 [1]
 [2]
 [2]
 [1]
 [1]
 [0]
 [0]
 [1]
 [0]
 [2]
 [1]
 [0]
 [2]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]
 [1]
 [2]
 [1]
 [1]
 [2]
 [1]
 [1]
 [0]
 [0]
 [1]
 [1]
 [1]]


In [24]:
from sklearn.metrics import f1_score
print('Macro f1_score = {}'.format(f1_score(y_dev, y_pred_final, average='macro')))
print('Micro f1_score = {}'.format(f1_score(y_dev, y_pred_final, average='micro')))


Macro f1_score = 0.3794703794703795
Micro f1_score = 0.6144578313253012


In [21]:
y_dev_id = dev_sub1_df.id

In [22]:
dict = {'article_id': y_dev_id, 'class': y_pred_sub} 
submit_df = pd.DataFrame(dict) 
submit_df.set_index('article_id', inplace=True)
submit_df.head()

,class
article_id,
820791520,opinion
821040551,reporting
813552066,opinion
817176202,opinion
820419869,reporting


In [ ]:
submit_df.to_csv('2_data_25_16.txt', sep='\t')

# Test Data

In [ ]:
test_df = pd.read_csv('/Users/nitanshjain/Documents/Projects/Sem_Eval/semeval2023task3/preprocessed_data/subtask1/en_test_subtask_1.csv')
test_df.head()

,Unnamed: 0,id,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,0,3121,The Pope Says 'Humanity Must Repent For Abusin...,We humans “must repent and modify our lifestyl...,the pope say humanity must repent for abuse mo...,"[(the, 'DT'), (pope, 'NN'), (say, 'VBP'), (hum...",[],we human must repent and modify our lifestyle ...,"[(we, 'PRP'), (human, 'NN'), (must, 'MD'), (re...","[(thursday, 'DATE', 391), (the world day, 'DAT..."
1,1,3135,Russian allies are unknowingly working against...,The Secretary of State for Defence described P...,russian ally be unknowingly work against they ...,"[(russian, 'JJ'), (ally, 'NN'), (be, 'VB'), (u...","[(russian, 'NORP', 381)]",the secretary of state for defence describe pu...,"[(the, 'DT'), (secretary, 'NNP'), (of, 'IN'), ...","[(putin, 'PERSON', 380), (russian, 'NORP', 381..."
2,2,3134,"I know this week has been disruptive, says LIZ...",The United Kingdom is the greatest country on ...,know this week have be disruptive say liz truss,"[(know, 'VB'), (this, 'DT'), (week, 'NN'), (ha...","[(this week, 'DATE', 391)]",the united kingdom be the great country on ear...,"[(the, 'DT'), (united, 'NNP'), (kingdom, 'NNP'...","[(the united kingdom, 'GPE', 384), (year, 'DAT..."
3,3,3120,Vladimir Putin facing Ukraine 'humiliation' as...,"VLADIMIR PUTIN is facing ""humiliation"" in Ukra...",vladimir putin face ukraine humiliation as rus...,"[(vladimir, 'NNP'), (putin, 'NNP'), (face, 'NN...","[(vladimir putin, 'PERSON', 380), (ukraine, 'G...",vladimir putin be face humiliation in ukraine ...,"[(vladimir, 'NNP'), (putin, 'NNP'), (be, 'VB')...","[(vladimir putin, 'PERSON', 380), (ukraine, 'G..."
4,4,3136,What Are the Odds Putin is Bluffing About Usin...,Eurointelligence founder Wolfgang Münchau has ...,what be the odd putin be bluff about use nucle...,"[(what, 'WP'), (be, 'VB'), (the, 'DT'), (odd, ...",[],eurointelligence founder wolfgang münchau have...,"[(eurointelligence, 'NN'), (founder, 'NN'), (w...","[(wolfgang münchau, 'PERSON', 380), (putin, 'P..."


In [ ]:
test_df.shape

(54, 10)

In [ ]:
x_test = test_df.preprocessed_headlines + test_df.preprocessed_articles
x_test

0     the pope say humanity must repent for abuse mo...
1     russian ally be unknowingly work against they ...
2     know this week have be disruptive say liz trus...
3     vladimir putin face ukraine humiliation as rus...
4     what be the odd putin be bluff about use nucle...
5     florida surgeon general issue warn for mrna co...
6     biden 's secret promise to opec backfire shell...
7     central bank add gold for fifth straight month...
8     we deny preposterous claim they sabotage nord ...
9     do uncle sam wile coyote blow up the wrong pip...
10    european health official admit microsoft found...
11    putin ally admit kyiv could soon bomb moscow a...
12    fda withhold autopsy result on people who die ...
13    russian plan reveal war with nato be comethis ...
14    al gore compare climate denier to police in uv...
15    dim witte duke prince harry slam for wade into...
16    american pravda of pipeline and plagueeurope b...
17    top un official we own the science and the

In [ ]:
x_test = roberta_encode(x_test, tokenizer)

In [ ]:
y_test = model_copy.predict(x_test)
y_test

2/2 [==============================] - 55s 22s/step


array([[0.3309119 , 0.33649427, 0.33259383],
       [0.33235344, 0.3341213 , 0.3335252 ],
       [0.33510315, 0.32963833, 0.3352585 ],
       [0.33401918, 0.33048645, 0.3354944 ],
       [0.33722687, 0.32900998, 0.33376315],
       [0.32986614, 0.33815804, 0.33197582],
       [0.3307315 , 0.33669588, 0.33257267],
       [0.32957992, 0.33848011, 0.33194003],
       [0.33234134, 0.3343677 , 0.33329096],
       [0.33614987, 0.32889634, 0.3349538 ],
       [0.33373123, 0.33119375, 0.33507496],
       [0.3302326 , 0.337745  , 0.33202237],
       [0.3299506 , 0.33806592, 0.33198354],
       [0.33686548, 0.32934746, 0.3337871 ],
       [0.33690703, 0.32874015, 0.3343528 ],
       [0.33456612, 0.32907277, 0.33636117],
       [0.3339    , 0.33140984, 0.33469012],
       [0.33432826, 0.32954812, 0.3361236 ],
       [0.33314273, 0.33312482, 0.3337325 ],
       [0.33153775, 0.3353226 , 0.33313972],
       [0.3287691 , 0.33932966, 0.33190125],
       [0.33095533, 0.3364902 , 0.33255446],
       [0.

In [ ]:
y_test_final = list()
y_test_sub = list()
for val in y_test:
    if val[0]>val[1] and val[0]>val[2]:
        y_test_final.append(0)
        y_test_sub.append('opinion')
    elif val[1]>val[0] and val[1]>val[2]:
        y_test_final.append(1)
        y_test_sub.append('reporting')
    else:
        y_test_final.append(2)
        y_test_sub.append('satire')

print(y_test_final) 

[1, 1, 2, 2, 0, 1, 1, 1, 1, 0, 2, 1, 1, 0, 0, 2, 2, 2, 2, 1, 1, 1, 0, 0, 1, 1, 0, 2, 0, 0, 0, 1, 1, 0, 1, 0, 0, 2, 2, 0, 2, 1, 0, 0, 2, 1, 0, 0, 1, 0, 2, 1, 1, 0]


In [ ]:
y_test_id = test_df.id

In [ ]:
dict_test = {'article_id': y_test_id, 'class': y_test_sub} 
submit_test_df = pd.DataFrame(dict_test) 
submit_test_df.set_index('article_id', inplace=True)
submit_test_df.head()

,class
article_id,
3121,reporting
3135,reporting
3134,satire
3120,satire
3136,opinion


In [ ]:
submit_test_df.to_csv('2_test_data_25_16.txt', sep='\t')